## Research Project 4
---
```text
- Source: SEC
- Goal: Extract CEO appointments from 8K
- Techniques: NER, sentence tokenization
- Tools: Spacy, NLTK
- Lines of code: ~50```

In [ ]:
# Standard library
import datetime

# Third-party
import spacy
import gensim
import requests
from lxml import html
from nltk.tokenize import sent_tokenize

url = 'https://www.sec.gov/Archives/edgar/daily-index/2018/QTR2/form.20180402.idx'
res = requests.get(url)
form_index = res.text.split('\n')
records = []
for line in form_index[11:]:
    record = {}
    record['filing'] = line[0:12].strip()
    record['name'] = line[12:74].strip()
    try:
        record['cik'] = int(line[74:86].strip())
    except ValueError:
        continue
    try:
        record['date'] = datetime.datetime(int(line[86:90]),
                                           int(line[90:92]),
                                           int(line[92:94]))
    except ValueError:
        continue
    record['path'] = 'https://www.sec.gov/Archives/' + line[98:].strip()
    records.append(record)

eight_ks = [i for i in records if i['filing'].startswith('8-K')]
for num, doc in enumerate(eight_ks):
    res = requests.get(doc['path'])
    tree = html.fromstring(res.content)
    clean = ' '.join(tree.itertext()).replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ')
    clean = re.sub(' +', ' ', clean)
    char = clean.find('Item 5.02')
    if char >= 0:
        sentences = sent_tokenize(clean[char:])
        joined = ' '.join(sentences[0:5])
        if 'CEO' in joined or 'Chief Executive Officer' in joined: 
            print('%d) %s: "%s"\n' % (num, doc['name'], joined))